In [17]:
import os
import pytz
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from django.apps import apps as django_apps
from edc_appointment.models import Appointment
from edc_visit_schedule.site_visit_schedules import site_visit_schedules

tz = pytz.timezone('Africa/Gaborone')

In [18]:
subject_identifier = 'B142-040990341-5'
child_identifier = 'B142-040990341-5-10'

enrol_onsch_obj = OnScheduleCohortBEnrollment.objects.get(
    subject_identifier=subject_identifier,
    child_subject_identifier=child_identifier)

quart_onsch_obj = OnScheduleCohortBQuarterly.objects.get(
    subject_identifier=subject_identifier,
    child_subject_identifier=child_identifier)

In [19]:
def get_cohort_schedule(schedule_type='enrollment'):
    current_cohort = Cohort.objects.get(subject_identifier=child_identifier, current_cohort=True)
    try:
        cohort_schedule = CohortSchedules.objects.get(cohort_name=current_cohort.name,
                                                      schedule_type=schedule_type,
                                                      onschedule_model__startswith='flourish_caregiver',
                                                      child_count=1, )
    except CohortSchedules.DoesNotExist:
        print('weh!')
    else:
        return cohort_schedule

In [8]:
cohort_schedule = get_cohort_schedule()

model_cls = django_apps.get_model(cohort_schedule.onschedule_model)
_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model=cohort_schedule.onschedule_model,
    name=cohort_schedule.schedule_name)

schedule.put_on_schedule(
    subject_identifier=subject_identifier,
    onschedule_datetime=enrol_onsch_obj.onschedule_datetime.astimezone(tz),
    schedule_name=cohort_schedule.schedule_name,
    base_appt_datetime=enrol_onsch_obj.onschedule_datetime.astimezone(tz))

In [9]:
enrol_appt = Appointment.objects.get(subject_identifier=subject_identifier,
                                     schedule_name=cohort_schedule.schedule_name)

enrol_visit = MaternalVisit.objects.get(subject_identifier=subject_identifier,
                                        schedule_name=enrol_onsch_obj.schedule_name)
old_appt = enrol_visit.appointment

enrol_appt.appt_datetime = old_appt.appt_datetime
enrol_appt.appt_type = old_appt.appt_type
enrol_appt.appt_status = old_appt.appt_status
enrol_appt.appt_reason = old_appt.appt_reason
enrol_appt.comment = old_appt.comment
enrol_appt.save()

enrol_visit.appointment = enrol_appt
enrol_visit.schedule_name = enrol_appt.schedule_name
enrol_visit.visit_schedule_name = enrol_appt.visit_schedule_name
enrol_visit.save()

In [20]:
cohort_schedule = get_cohort_schedule(schedule_type='quarterly')

model_cls = django_apps.get_model(cohort_schedule.onschedule_model)
_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model=cohort_schedule.onschedule_model,
    name=cohort_schedule.schedule_name)

schedule.put_on_schedule(
    subject_identifier=subject_identifier,
    onschedule_datetime=quart_onsch_obj.onschedule_datetime.astimezone(tz),
    schedule_name=cohort_schedule.schedule_name,
    base_appt_datetime=quart_onsch_obj.onschedule_datetime.astimezone(tz))

In [22]:
quart_appts = Appointment.objects.filter(subject_identifier=subject_identifier,
                                         schedule_name=cohort_schedule.schedule_name)

quart_visits = MaternalVisit.objects.filter(subject_identifier=subject_identifier,
                                            schedule_name=quart_onsch_obj.schedule_name)

for quart_visit in quart_visits:
    old_appt = quart_visit.appointment
    quart_appt = quart_appts.get(
        visit_code=old_appt.visit_code,
        visit_code_sequence=old_appt.visit_code_sequence)

    quart_appt.appt_datetime = old_appt.appt_datetime
    quart_appt.appt_type = old_appt.appt_type
    quart_appt.appt_status = old_appt.appt_status
    quart_appt.appt_reason = old_appt.appt_reason
    quart_appt.comment = old_appt.comment
    quart_appt.save()

    quart_visit.appointment = quart_appt
    quart_visit.schedule_name = quart_appt.schedule_name
    quart_visit.visit_schedule_name = quart_appt.visit_schedule_name
    quart_visit.save()

In [23]:
def delete_old_schedules(onschedule_obj):
    appts = Appointment.objects.filter(
        subject_identifier=onschedule_obj.subject_identifier,
        schedule_name=onschedule_obj.schedule_name)
    ssh = SubjectScheduleHistory.objects.filter(
        subject_identifier=onschedule_obj.subject_identifier,
        schedule_name=onschedule_obj.schedule_name)
    appts.delete()
    ssh.delete()
    onschedule_obj.delete()

delete_old_schedules(enrol_onsch_obj)
delete_old_schedules(quart_onsch_obj)

In [24]:
quart_appts.update(appt_status='incomplete')

15